In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
main_data_dir = '/kaggle/input/garbage-classification/garbage_classification'

In [3]:
train_dir = '/kaggle/working/train'
test_dir = '/kaggle/working/test'
val_dir = '/kaggle/working/valid'

In [4]:
class_directories = [d for d in os.listdir(main_data_dir) if os.path.isdir(os.path.join(main_data_dir, d))]


In [5]:
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [6]:
for class_dir in class_directories:
    class_path = os.path.join(main_data_dir, class_dir)
    
    # Get the list of all images in the class directory
    all_images = [img for img in os.listdir(class_path) if img.endswith('.jpg')]
    
    # Split the data into train, test, and validation sets
    train_images, test_val_images = train_test_split(all_images, test_size=0.3, random_state=42)
    test_images, val_images = train_test_split(test_val_images, test_size=0.5, random_state=42)
    
    for img in train_images:
        src = os.path.join(class_path, img)
        dest = os.path.join(train_dir, class_dir, img)
        os.makedirs(os.path.dirname(dest), exist_ok=True)
        shutil.copy(src, dest)
    
    for img in test_images:
        src = os.path.join(class_path, img)
        dest = os.path.join(test_dir, class_dir, img)
        os.makedirs(os.path.dirname(dest), exist_ok=True)
        shutil.copy(src, dest)
    
    for img in val_images:
        src = os.path.join(class_path, img)
        dest = os.path.join(val_dir, class_dir, img)
        os.makedirs(os.path.dirname(dest), exist_ok=True)
        shutil.copy(src, dest)

In [7]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, VGG16, InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Image dimensions
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)

# Data augmentation for training set
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Generate batches of augmented data for training and validation
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(img_width, img_height), batch_size=32, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(img_width, img_height), batch_size=32, class_mode='categorical')


# Generate batches of augmented data for testing
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(img_width, img_height), batch_size=32, class_mode='categorical')


Found 10854 images belonging to 12 classes.
Found 2333 images belonging to 12 classes.
Found 2328 images belonging to 12 classes.


In [8]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
cnn_model = Sequential()
cnn_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(12, activation='softmax'))

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(train_generator, epochs=20, validation_data=val_generator)

Epoch 1/20
340/340 [==============================] - 132s 344ms/step - loss: 1.7242 - accuracy: 0.4979 - val_loss: 1.2753 - val_accuracy: 0.6074
Epoch 2/20
340/340 [==============================] - 117s 344ms/step - loss: 1.1145 - accuracy: 0.6282 - val_loss: 1.0127 - val_accuracy: 0.6687
Epoch 3/20
340/340 [==============================] - 117s 343ms/step - loss: 0.9371 - accuracy: 0.6860 - val_loss: 0.9861 - val_accuracy: 0.6734
Epoch 4/20
340/340 [==============================] - 119s 350ms/step - loss: 0.8328 - accuracy: 0.7207 - val_loss: 0.9030 - val_accuracy: 0.7201
Epoch 5/20
340/340 [==============================] - 116s 341ms/step - loss: 0.7605 - accuracy: 0.7470 - val_loss: 1.0639 - val_accuracy: 0.6794
Epoch 6/20
340/340 [==============================] - 118s 346ms/step - loss: 0.6864 - accuracy: 0.7746 - val_loss: 0.9038 - val_accuracy: 0.7235
Epoch 7/20
340/340 [==============================] - 118s 347ms/step - loss: 0.6309 - accuracy: 0.7924 - val_loss: 0.8411 -

In [9]:
cnn_model.evaluate(test_generator)

73/73 [==============================] - 4s 57ms/step - loss: 1.0851 - accuracy: 0.7607


[1.0851374864578247, 0.7607388496398926]